# Models metrics

In [40]:
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from numpy.ma.core import equal
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from statsmodels.stats.weightstats import ttest_ind
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [41]:
df = pd.read_csv('/kaggle/input/ninput/train.csv')
df.replace('unknown', np.nan, inplace=True)
X_train = df.iloc[:, 1:-1]
y_train = df['y']
df_test = pd.read_csv('/kaggle/input/ninput/test.csv')
df_test.replace('unknown', np.nan, inplace=True)
X_test = df_test.iloc[:, 1:]

In [42]:
a = [1 if x == 1 else 0 for x in y_train]
print(sum(a))
b = [1 if x == 0 else 0 for x in y_train]
sum(b)
print(sum(b) / sum(a))

90488
7.288391830961012


In [43]:
cat_features = X_train.select_dtypes(include='object').columns.to_list()

In [44]:
from category_encoders import CatBoostEncoder

preprocessor = ColumnTransformer([('enc', OneHotEncoder(), cat_features)], remainder='passthrough')

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import svm
xgb_pipe = Pipeline([('preprocessor', preprocessor), ('classifier', XGBClassifier(scale_pos_weight=7.29))])
#xgb_pipe = Pipeline([('preprocessor', preprocessor), ('classifier', CatBoostClassifier(learning_rate=0.3 ,depth=5, iterations=408, verbose=False) )])
#xgb_pipe.fit(X_train, y_train)

In [ ]:
#cross_val_score(xgb_pipe, X_train, y_train, cv=0, scoring='roc_auc', n_jobs=-1).mean()


In [ ]:
'''
param_grid = {
    'classifier__n_estimators': [400, 600, 800, 1000],
    'classifier__max_depth': [5, 6, 7, 8],
     'classifier__learning_rate' : [0.06, 0.08, 0.1]

}

grid_search = GridSearchCV(
    estimator=xgb_pipe,
    param_grid = param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
results_df = pd.DataFrame(grid_search.cv_results_)
results_df.to_csv('/kaggle/working/grid_search_results.csv', index=False)
best_model = grid_search.best_estimator_
results_df.head(20)
'''

In [ ]:
#print(#cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1).mean())

In [ ]:
#grid_search.best_estimator_['classifier'].save_model('model')

In [ ]:
preprocessor = ColumnTransformer([('enc', OneHotEncoder(), cat_features)], remainder='passthrough')

In [45]:
pipe = Pipeline([('preprocessor', preprocessor), ('classifier', XGBClassifier(learning_rate=0.1, n_estimators=1000, max_depth=5, scale_pos_weight=7.29))])
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc', OneHotEncoder(),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'poutcome'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=Non...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=1000, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [ ]:
#print(cross_val_score(pipe, X_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1).mean())

In [49]:
df_submission = pd.read_csv('/kaggle/input/ninput/sample_submission.csv', index_col=False)
df_submission['y'] = pipe.predict_proba(X_test)[:, 1]
df_submission.columns = ['id', 'y']
df_submission.to_csv('submission.csv', index=False)